# Hand-tuning a (simple) model

A neural network is just a mathematical function. "Learning" or "tuning" means adjusting the function so that it maps inputs to the desired outputs.

## Simple Example: Learning a Quadratic Function

But a neural network is a rather complex mathematical function. To get a better handle on the method of tuning a function, we'll start with a (much) simpler example, a quadratic function. We're going to:
  1. start with a function
  2. using our starting function, simulate some "noisy" data,
  3. and then try to **learn** a function that fits this noisy data.

First, a little setup. I'm importing some libraries and creating a utility to graph functions.

In [ ]:
from ipywidgets import interact
from fastai.basics import *

def plot_function(f, title=None, min=-2.1, max=2.1, color='r', ylim=None):
    x = torch.linspace(min,max, 100)[:,None]
    if ylim: plt.ylim(ylim)
    plt.plot(x, f(x), color)
    if title is not None: plt.title(title)

Here's our "known" function:

In [ ]:
def f(x): 
    return 3*x**2 + 2*x + 1

plot_function(f, "$3x^2 + 2x + 1$", min=-4.1, max=4.1)

This quadratic is of the form $ax^2+bx+c$, with parameters $a=3$, $b=2$, $c=1$. To make it easier to try out different quadratics for fitting a model to the data we'll create, let's create a function that calculates the value of a point on any quadratic:

In [ ]:
# general quadratic function f(x) with parameters a, b, c
def quad(a, b, c, x): 
    return a*x**2 + b*x + c

If we fix some particular values of a, b, and c, then we'll have created a specific quadratic. 
To fix values passed to a function in python, we use the `partial` function, like so:

In [ ]:
# factory to create parameterized quadratics
def mk_quad(a,b,c):
    # returns a function that has a,b,c set, only takes remaining x parameter
    return partial(quad, a,b,c)

For instance, we can recreate our previous quadratic:

In [ ]:
f2 = mk_quad(3,2,1) # make a quatratic function that has the parameters a=3, b=2, c=1
plot_function(f2)

Now let's simulate making some noisy measurements of our quadratic *f(x)*. We'll then use gradient descent to see if we can recreate the original function from the data.

Here's a couple of functions to add some random noise to data:

In [ ]:
def noise(x, scale): return tensor(np.random.normal(scale=scale, size=x.shape))
def add_noise(x, mult, add): return x * (1+noise(x,mult)) + noise(x,add)

Let's use the now to create our noisy measurements based on the quadratic above:

In [ ]:
np.random.seed(42)

x = torch.linspace(-2, 2, steps=20)[:,None] # the x-axis (independent) values (20 evenly spaced between -2 and 2)
y = add_noise(f(x), 0.15, 1.2) # the y-axis (dependent) values [f(x) +- some random noise]

In [ ]:
plt.scatter(x,y);

How do we find values of a, b, and c which fit this data? One approach is to try a few values and see what fits. Here's a function which overlays a quadratic on top of our data, along with some sliders to change a, b, and c, and see how it looks:

In [ ]:
@interact(a=1.1, b=1.1, c=1.1)
def plot_quad(a, b, c):
    plt.scatter(x,y)
    plot_function(mk_quad(a,b,c), ylim=(-3,13))

Move the slider for `a` to get the best fit possible. Then, adjust `b` and after that `c`.

OK, now go back to slider `a` and repeat the process. Do it again for `b` and `c` as well.

Did you notice that by going back and doing the sliders a second time that you were able to improve things a bit further? That's an important insight -- it's only after changing `b` and `c`, for instance, that you realise that `a` actually needs some adjustment based on those new values.

One thing that's making this tricky is that we don't really have a great sense of whether our fit is really better or worse. It would be easier if we had a numeric measure of that. On easy metric we could use is *mean absolute error* -- which is the distance from each data point to the curve:

In [ ]:
def mae(preds, acts): return (torch.abs(preds-acts)).mean()

We'll update our interactive function to print this at the top for us. Try fitting the line again, this time with the help of the mean absolute error measurement.

You'll probably find that you have to go through the set of sliders a couple of times to get the best fit.

In [ ]:
@interact(a=1.1, b=1.1, c=1.1)
def plot_quad(a, b, c):
    f = mk_quad(a,b,c)
    plt.scatter(x,y)
    loss = mae(f(x), y)
    plot_function(f, ylim=(-3,12), title=f"MAE: {loss:.2f}")

Got it dialed in pretty well? Congrats just hand-tuned a model. 

But in a modern neural network we'll often have tens of millions of parameters to fit, or more, and thousands or millions of data points to fit them to. We're not going to be able to do that by moving sliders around! We'll need to automate this process. That's what we'll do in the next notebook.